# Join two dataframes and load the output as a new table in the database

In [1]:
# Dependencies 

## To access the MySQL database
from sqlalchemy import create_engine
from config import password

## In case pymysql is not yet installed in the local station 
import pymysql
pymysql.install_as_MySQLdb()

## For data processing
import pandas as pd

In [2]:
# Create a connection to the database
conn = "root:{0}@localhost:3306/diabetes_db".format(password) # Password is in a separate file
engine = create_engine(f"mysql://{conn}")

In [3]:
# Confirm presence of tables
engine.table_names()

/anaconda3/envs/PythonData/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


['diabetes', 'merged', 'obesity']

## Extract data from the diabetes database

In [4]:
# Read the table contents (for diabetes)
diabetes = pd.read_sql("select * from diabetes", con = engine)

# Preview the table contents
diabetes.head()

,Year,US_State,Adult_Diabetics_Percent
0,2012,Guam,9.6
1,2012,Alaska,7
2,2012,Alabama,12.2
3,2012,Arkansas,11.3
4,2012,Arizona,10.6


In [5]:
# Read the table contents (for obesity)
obesity = pd.read_sql("select * from obesity", con = engine)

# Preview the table contents
obesity.head()

,Year,US_State,Obese_Children_Percent
0,2010,Alabama,15.8
1,2012,Alabama,15.6
2,2014,Alabama,16.3
3,2010,Alaska,21.2
4,2012,Alaska,20.6


## Transform the data (merging dataframes)

In [6]:
# Merge diabetes and obesity dataframes
merged = pd.merge(diabetes, obesity, on=['Year','US_State'], how='inner')
    # inner join is used to retain only the contents that have common year & 
    # US state combinations

# Preview the new data frame
merged.head()

,Year,US_State,Adult_Diabetics_Percent,Obese_Children_Percent
0,2012,Guam,9.6,10.0
1,2012,Alaska,7,20.6
2,2012,Alabama,12.2,15.6
3,2012,Arkansas,11.3,14.6
4,2012,Arizona,10.6,14.9


## Load the data as a new table in the diabetes database

In [7]:
# Load the "merged" dataframe into the diabetes database in a table called "merged"
merged.to_sql(name = "merged", con = engine, if_exists = "replace", index = False)
    # if_exists = "replace" ensures that when the code is run again, the existing database 
    # table won't have duplicate copies

In [8]:
# Preview the contents of the new database table, "merged"
pd.read_sql_query("select * from merged", con = engine).head()

,Year,US_State,Adult_Diabetics_Percent,Obese_Children_Percent
0,2012,Guam,9.6,10.0
1,2012,Alaska,7,20.6
2,2012,Alabama,12.2,15.6
3,2012,Arkansas,11.3,14.6
4,2012,Arizona,10.6,14.9
